#### 🔹 Import Necessary Libraries

In [145]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [146]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies = movies[['id','title','overview','genres', 'homepage']]
movies["overview"]= movies["overview"].fillna("")
movies.head()

,id,title,overview,genres,homepage
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter


In [147]:
import ast

def clean_genres(genre_str):
    genres_list = ast.literal_eval(genre_str)  # Convert JSON string to list
    return " ".join([genre["name"] for genre in genres_list])

movies['genres'] = movies['genres'].apply(clean_genres)
movies.head()

,id,title,overview,genres,homepage
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,http://www.thedarkknightrises.com/
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,http://movies.disney.com/john-carter


In [148]:
movies["Content"] = movies["title"] +" "+ movies["genres"] +" "+ movies["overview"]
movies.head()

,id,title,overview,genres,homepage,Content
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,Avatar Action Adventure Fantasy Science Fictio...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End Adven...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,Spectre Action Adventure Crime A cryptic messa...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,The Dark Knight Rises Action Crime Drama Thril...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,http://movies.disney.com/john-carter,John Carter Action Adventure Science Fiction J...


In [149]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["Content"])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [150]:
def recommend(movie_title):
    try:
        idx_list = movies[movies["title"] == movie_title].index.tolist()
        if not idx_list:
            return ["Movie not found"]

        idx = idx_list[0]
        scores = list(enumerate(cosine_sim[idx]))  
        scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:6]  

        recommendations = [
            {"title": movies.iloc[i[0]]["title"], "homepage": movies.iloc[i[0]]["homepage"]}
            for i in scores
        ]
        return recommendations   
    except Exception as e:
        return [f"Error: {str(e)}"]

In [151]:
print(recommend("friends"))

['Movie not found']


In [152]:
import streamlit as st

st.title("Movie Recommendation System 🎬")

movie_name = st.text_input("Enter a movie name:")
if st.button("Get Recommendations"):
    recommendations = recommend(movie_name)

    if isinstance(recommendations, list) and "Movie not found" in recommendations:
        st.error("Movie not found. Try another title.")
    else:
        for movie in recommendations:
            st.subheader(movie["title"])
            st.markdown(f"[Visit Homepage]({movie['homepage']})", unsafe_allow_html=True)

2025-03-30 03:28:36.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-30 03:28:36.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run main.py --server.headless true